The aim of this notebook is to determine the survival rate of technologies in files after an other technology has been introduced

In [3]:
load.ivy("joda-time" % "joda-time" % "2.8")
load.ivy("com.github.nscala-time" %% "nscala-time" % "2.2.0")

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT from sonatype-snapshots, using Sun Oct 25 19:59:24 CET 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Wed Oct 21 11:43:36 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT from sonatype-snapshots, using Wed Oct 21 17:03:52 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT from sonatype-snapshots, using Sun Oct 25 19:59:24 CET 2015

	Choosing sonatype-

In [4]:
import java.io.File
import org.joda.time.{DateTime, DateTimeZone, Duration, Interval, Days, Weeks, Months}
import org.joda.time.format.DateTimeFormat
import scala.io.Source
import java.io.{PrintWriter,FileOutputStream}
import com.github.nscala_time.time.Imports._

import java.io.File
import org.joda.time.{DateTime, DateTimeZone, Duration, Interval, Days, Weeks, Months}
import org.joda.time.format.DateTimeFormat
import scala.io.Source
import java.io.{PrintWriter,FileOutputStream}
import com.github.nscala_time.time.Imports._

In [5]:
val directory = new File("/Users/mg/CloudStation/publications/2016/saner2016")
val first_last_file = new File(directory, "data/first-last.summary.csv")
val project_summary_file = new File(directory, "data/projects.summary.csv")
val dtf = DateTimeFormat.forPattern("yyyy-MM-dd HH:mm:ss").withZoneUTC

directory: File = /Users/mg/CloudStation/publications/2016/saner2016
first_last_file: File = /Users/mg/CloudStation/publications/2016/saner2016/data/first-last.summary.csv
project_summary_file: File = /Users/mg/CloudStation/publications/2016/saner2016/data/projects.summary.csv
dtf: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@a9f227f

In [6]:
val project_end = Source.fromFile(project_summary_file)
                        .getLines.drop(1)
                        .map(l => l.split(","))
                        .map(l => l(0) -> dtf.withZoneUTC.parseDateTime(l.last.substring(0,19)))
                        .toMap

project_end: Map[String, DateTime] = Map(
  "Netflix_astyanax" -> 2015-01-26T12:52:19.000Z,
  "stewartsims_KMS-dev" -> 2011-08-12T15:22:33.000Z,
  "limcheekin_vaadin-addressbook-sample" -> 2012-06-18T10:49:00.000Z,
  "apache_maven-plugins" -> 2015-03-18T22:11:07.000Z,
  "gphat_bullfinch" -> 2012-06-26T11:15:32.000Z,
  "sagioto_forum" -> 2012-07-30T18:12:20.000Z,
  "jpaoletti_java-presentation-manager" -> 2013-06-25T19:57:22.000Z,
  "qyb_sohu" -> 2012-03-30T16:17:54.000Z,
  "ebayopensource_turmeric-releng" -> 2012-01-12T09:44:28.000Z,
  "restsql_restsql" -> 2015-02-16T14:46:20.000Z,
  "openshift_kitchensink-example" -> 2014-03-31T10:02:07.000Z,
  "hamnis_httpcache4j" -> 2015-02-25T23:01:51.000Z,
  "Tagette_BossStory" -> 2014-12-22T16:07:46.000Z,
  "eclipse_cdo" -> 2015-03-18T08:05:25.000Z,
...

In [7]:
implicit def orderingByDateTime[A <: DateTime]: Ordering[A] = Ordering.by(d => d.getMillis)

case class Row(project: String, 
               file: String,
               file_end: DateTime,
               project_end: DateTime,
               technologies: Map[String, Interval])
{
    def is_entity_related(): Boolean = !technologies.isEmpty
    
    def involves(technology: String): Boolean = technologies.keySet contains technology
    
    def involves(technology: String, date: DateTime): Boolean = involves(technology) && (technologies(technology) contains date)
    
    def is_jpa(t: DateTime): Boolean = technologies.keySet contains "JPA"
    
    def is_jdbc(t: DateTime): Boolean = technologies.keySet contains "JDBC"
    
    def is_hbm(t: DateTime): Boolean = technologies.keySet contains "Hibernate"
}

object Row
{
    def apply(line: String, project_end: Map[String, DateTime]) = {
        val elements = line.split(",", -1)
        val project = elements(0)
        val file = elements(1)
        val jpa = elements(2) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval( dtf.parseDateTime(elements(2)) , dtf.parseDateTime(elements(3))))  
        }
        
        val jdbc = elements(4) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval(dtf.parseDateTime(elements(4)) , dtf.parseDateTime(elements(5))))  
        }
        
        val hbm = elements(6) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval(dtf.parseDateTime(elements(6)) , dtf.parseDateTime(elements(7))))  
        }
        
        val technologies = scala.collection.mutable.Map[String, Interval]()
        if(jpa.isDefined) technologies.put("JPA", jpa.get)
        if(jdbc.isDefined) technologies.put("JDBC", jdbc.get)
        if(hbm.isDefined) technologies.put("Hibernate",hbm.get)
        
        if(project_end.keySet contains project) Some(new Row(project,file,project_end(project),project_end(project),technologies.toMap))
        else None
    }
}

defined function orderingByDateTime
defined class Row
defined object Row

In [8]:
val entries = Source.fromFile(first_last_file)
                    .getLines.drop(1)
                    .filterNot(_.trim.isEmpty)
                    .map(l => Row.apply(l, project_end))
                    .toList
                    .flatten

entries: List[Row] = List(
  Row(
    "krinsdeath_JobSuite",
    "src/main/java/net/krinsoft/jobsuite/JobManager.java",
    2013-05-07T11:08:50.000Z,
    2013-05-07T11:08:50.000Z,
    Map("JDBC" -> 2012-06-11T23:59:13.000Z/2013-05-07T11:08:50.000Z)
  ),
  Row(
    "krinsdeath_JobSuite",
    "src/main/java/net/krinsoft/jobsuite/db/Database.java",
    2013-05-07T11:08:50.000Z,
    2013-05-07T11:08:50.000Z,
    Map("JDBC" -> 2012-06-11T23:59:13.000Z/2013-05-07T11:08:50.000Z)
  ),
...

Release the survival data of a sequence of values.
Each value is a duration, plus a boolean expressing if the event actually occured

In [8]:
println("Number of files: " + entries.size)
println("Number of projects: " + project_end.size)

Number of files: 112261
Number of projects: 2457


In [9]:
def minDate(a: DateTime, b: DateTime): DateTime = if(a isBefore b) a else b
def maxDate(a: DateTime, b: DateTime): DateTime = if(a isAfter b) a else b 

defined function minDate
defined function maxDate

In [10]:
def release_survival(values: Seq[(Int, Boolean)], file: File) =
{
    val out = new PrintWriter(file)
    out.println("duration,event")
    values.foreach(value => {
        out.println(value._1 + "," + (if(value._2) 1 else 0))
    })
    
    out.flush()
    out.close()
}

defined function release_survival

Generate survival files

In [11]:
val technologies = Seq("JPA", "JDBC", "Hibernate")

technologies: Seq[String] = List("JPA", "JDBC", "Hibernate")

In [194]:
technologies.foreach(t1 => {
    technologies.foreach(t2 => {
        if(t1 != t2)
        {
            val candidates = entries.filter(p => (p.technologies.keySet contains t1) && 
                                                 (p.technologies.keySet contains t2) && 
                                                !(p.technologies(t1).getStart isAfter p.technologies(t2).getStart) &&
                                                 
                                                 (p.technologies(t1).getEnd isAfter p.technologies(t2).getStart)
                                           )
            
            val values = candidates.map(candidate => {
                val duration = Days.daysBetween(candidate.technologies(t2).getStart,
                                                candidate.technologies(t1).getEnd)
                                   .getDays
                assert(duration >= 0)
                val event = candidate.technologies(t1).getEnd isBefore project_end(candidate.project)
                
                (duration, event)
            })
            
            println(t1 + " -> " + t2 + " : " + candidates.size + " files" )
            release_survival(values, new File(directory, "data/survival-" + 
                                              t1.toLowerCase + "-" + t2.toLowerCase + ".csv"))
        }
    })
})

JPA -> JDBC : 801 files
JPA -> Hibernate : 263 files
JDBC -> JPA : 769 files
JDBC -> Hibernate : 181 files
Hibernate -> JPA : 177 files
Hibernate -> JDBC : 207 files


Determines the time between t2 introduction and t1 disappearance.

This time will be negative if t1 is removed before t2 is introduced.

In [195]:
technologies.foreach(t1 => {
    technologies.foreach(t2 => {
        if(t1 != t2)
        {
            val candidates = entries.filter(p => (p involves t1) && 
                                                 (p involves t2) && 
                                                !(p.technologies(t1).getStart isAfter p.technologies(t2).getStart))
            
            val values = candidates.map(candidate => {
                val duration = Days.daysBetween(candidate.technologies(t2).getStart,
                                                candidate.technologies(t1).getEnd)
                                   .getDays
                val event = candidate.technologies(t1).getEnd isBefore project_end(candidate.project)
                
                (duration, event)
            })
            
            println(t1 + " -> " + t2 + " : " + candidates.size + " files" )
            release_survival(values, new File(directory, "data/survival-" + 
                                              t1.toLowerCase + "-" + t2.toLowerCase + ".csv"))
        }
    })
})

JPA -> JDBC : 869 files
JPA -> Hibernate : 283 files
JDBC -> JPA : 839 files
JDBC -> Hibernate : 188 files
Hibernate -> JPA : 379 files
Hibernate -> JDBC : 218 files


Retrieves the first and last date a technology occured in a project

In [12]:
def limit_dates(project: String, entries: List[Row], technology: String): Option[Interval] = 
{
   val dates = entries.filter(e => e.project == project)
                      .map(e => e.technologies.get(technology))
                      .flatten
    
   if(dates.isEmpty) None
   else Some(dates.reduce((a,b) => {
              new Interval( minDate(a.getStart, b.getStart), maxDate(a.getEnd, b.getEnd))
          }))
}

defined function limit_dates

Determines the files having A when B has been introduced in the project, and finally having B
Duration: number of days between the introduction of B in the project, and the moment when B is introduced in the file
Event: occurs if B is finally introduced in the considered files

In [197]:
technologies.foreach(t2 => {
    val interesting_projects = project_end.keySet filter (p => limit_dates(p, entries, t2).isDefined)
    
   
    technologies.foreach(t1 => {
        if(t1 != t2)
        {
           val values = interesting_projects.toSeq.map(project => {
               val project_t2_start = limit_dates(project, entries, t2).get.getStart
               val interesting_files = entries.filter(entry => (entry.project == project) && 
                                                               (entry involves t1) && 
                                                               (entry.technologies(t1) contains project_t2_start)   
                                                      )
               interesting_files.map(file => {
                   if(file involves t2){
                       val duration = Days.daysBetween(project_t2_start, file.technologies(t2).getStart)
                                        .getDays 
                       (duration, true)
                   }
                   else
                   {
                       val duration = Days .daysBetween(project_t2_start, file.file_end)
                                           .getDays
                       (duration, false)
                   }
               })
           }).flatten
            
           println(t1 + " -> " + t2 + " : " + values.size + " values to register")
            
           release_survival(values, new File(directory, "data/survival-" + 
                                              t1.toLowerCase + "-after-" + t2.toLowerCase + ".csv"))   
        
           
        }
    })
})

JDBC -> JPA : 7444 values to register
Hibernate -> JPA : 1350 values to register
JPA -> JDBC : 3554 values to register
Hibernate -> JDBC : 1656 values to register
JPA -> Hibernate : 1059 values to register
JDBC -> Hibernate : 2154 values to register


Determines the proportion of files having A when B is introduced in a project, and that finished by having B.

In [198]:
val out = new PrintWriter(new File(directory, "data/contamination-ratio.csv"))
out.println("project,t1,t2,ratio")
    
technologies.foreach(t2 => {
    val interesting_projects = project_end.keySet filter (p => limit_dates(p, entries, t2).isDefined)
    
   
    technologies.foreach(t1 => {
        if(t1 != t2)
        {
            val ratios = interesting_projects.toSeq.map(project => {
               val project_t2_start = limit_dates(project, entries, t2).get.getStart

               val interesting_files = entries.filter(entry => (entry.project == project) && 
                                                               (entry involves t1) && 
                                                               (entry.technologies(t1) contains project_t2_start))   
                
               val converted_files = interesting_files.filter(file => (file involves t2))
                
               if(interesting_files isEmpty) None
               else Some(project, (converted_files.size.toFloat / interesting_files.size.toFloat)) 
            })
            
            ratios.flatten.foreach(ratio => {
                out.println(ratio._1 + "," + t1 + "," + t2 + "," + ratio._2)
            })
            
        }
    })
})

out.close()

out: PrintWriter = java.io.PrintWriter@4468f2c5

# Contamination evolution

Determines the proportion of files having A when B has been introduced in a project, and that finish by having B before a given duration

In [199]:
val out = new PrintWriter(new File(directory, "data/contamination-ratio-evolution.csv"))
out.println("project,t1,t2,duration,ratio,cumratio")
val MAX_WEEK = 15

technologies.foreach(t2 => {
    val interesting_projects = project_end.keySet.filter (p => limit_dates(p, entries, t2).isDefined)

    technologies.foreach(t1 => {
        if(t2 != t1)
        {
            val weeks = interesting_projects.toSeq.map(project => {
                val project_t2_start = limit_dates(project, entries, t2).get.getStart
                val interesting_files = entries.filter(entry =>(entry.project == project) && 
                                                               (entry involves t1) && 
                                                               (entry.technologies(t1) contains project_t2_start))   
           
                val durations = interesting_files.map(file => {
                    if(!file.involves(t2)) None
                    else Some(Weeks.weeksBetween(project_t2_start, file.technologies(t2).getStart).getWeeks)
                }) 
                
                project -> (interesting_files.size, durations.flatten)
            }).toMap 
                       
            weeks.filterNot(w => w._2._1 == 0).foreach(w => {
                val project = w._1
                val nb_files = w._2._1.toFloat
                val durations = w._2._2
                
                (0 to MAX_WEEK).foreach(week => {
                    val ratio = durations.count(d => d == week).toFloat / nb_files.toFloat
                    val cumRatio = durations.count(d => d <= week).toFloat / nb_files.toFloat
                    out.println(project + "," + t1 + "," + t2 + "," + week + "," + ratio + "," + cumRatio)
                })
            })
        }
    })
})

out.close

out: PrintWriter = java.io.PrintWriter@45cc20cd
MAX_WEEK: Int = 15

# Ratio evolution

Determines the proportion of files having A when B has been introduced in a project, and that 
 - don't have neither A nor B (suppression)
 - have A and B (completion)
 - don't have A and have B (replacement)
 - have A and don't have B (residual)
 
These proportions are evaluated over time, for predefined weeks following the introduction of B in the project.
 
Only projects in which A has been introduced at least 1 week before B are used.
If a project finished before one of the considered time, there is no output for this project and this time.

In [200]:
val out = new PrintWriter(new File(directory, "data/partition-ratio-evolution.csv"))
out.println("project,interestingfiles,t1,t2,duration,suppression,completion,replacement,residual")
val interesting_weeks = Seq(0,1,8,15)
val technologies = Seq("JPA", "Hibernate")

technologies.foreach(t2 => {
    val interesting_projects = project_end.keySet.filter (p => limit_dates(p, entries, t2).isDefined)

    technologies.foreach(t1 => {
        if(t1 != t2)
        {   
                val selected_projects = interesting_projects.filter(project => {
                val res_t2 = limit_dates(project, entries, t2).get
                val project_t2_start = res_t2.getStart
                val project_t2_end = res_t2.getEnd
                                    
                val res_t1 = limit_dates(project, entries, t1)
                val project_t1_start = res_t1.map(_.getStart)
                val project_t1_end = res_t1.map(_.getEnd)
   
                // Filter projects in which A has been introduced at least 1 week before B,
                // and B has been introduced while A was still present
                project_t1_start.isDefined && 
                (!(project_t1_start.get isAfter (project_t2_start minus Weeks.ZERO ))) && 
                (project_t2_start isBefore project_t1_end.get)
            })
            
            println(t1 + " -> " + t2 + " : " + selected_projects.size + " selected projects")
            
            selected_projects.foreach(project => {
                val project_t2_start = limit_dates(project, entries, t2).get.getStart
                val interesting_files = entries.filter(file => (file.project == project) && 
                                                               (file involves t1) && 
                                                               (file.technologies(t1) contains project_t2_start))
                
                val file_size = interesting_files.size.toFloat
                
                val global_end = project_end(project)
                
                interesting_weeks.foreach(week => {
                    val date = project_t2_start plus Weeks.weeks(week)
                    
                    if(date isBefore global_end) // The project is not finished
                    {
                        val suppression = interesting_files.count(file => (!file.involves(t1, date)) && (!file.involves(t2, date)))
                        val suppression_ratio = suppression.toFloat / file_size

                        val completion  = interesting_files.count(file => file.involves(t1, date) && file.involves(t2, date))
                        val completion_ratio = completion.toFloat / file_size

                        val replacement = interesting_files.count(file => (!file.involves(t1, date)) && file.involves(t2, date))
                        val replacement_ratio = replacement.toFloat / file_size

                        val residual    = file_size - suppression - completion - replacement
                        val residual_ratio = residual.toFloat / file_size                                          

                        assert(residual >= 0.0)

                        out.println(project + "," + interesting_files.size + "," + 
                                    t1 + "," + t2 + "," + week + "," +
                                    suppression_ratio + "," + completion_ratio + "," + 
                                    replacement_ratio + "," + residual_ratio)  
                    }
 
                })

            })
        }
    })
})
        
out.close()


Hibernate -> JPA : 48 selected projects
JPA -> Hibernate : 42 selected projects


out: PrintWriter = java.io.PrintWriter@36f6a2c5
interesting_weeks: Seq[Int] = List(0, 1, 8, 15)
technologies: Seq[String] = List("JPA", "Hibernate")

# Number of files and projects having each of the technologies, over time

In [37]:
val first_date = entries.flatMap(e => e.technologies.map(t => t._2.getStart)).minBy(date => date.getMillis)
val last_date = entries.flatMap(e => e.technologies.map(t => t._2.getEnd)).maxBy(date => date.getMillis)

val out = new PrintWriter(new File(directory, "data/techno-usage-evolution.csv"))
var current = first_date
val dtf_short = DateTimeFormat.forPattern("yyyy-MM");
out.println("date,jpa-files,jdbc-files,hibernate-files,jpa-projects,jdbc-projects,hibernate-projects")
while(current isBefore last_date)
{
    val results_files = technologies.map(techno => {
        techno -> entries.count(entry => (entry involves techno) && (entry.technologies(techno) contains current))
    }).toMap
    
    val results_projects = technologies.map(techno => {
        techno -> entries.filter(entry => (entry involves techno) && (entry.technologies(techno) contains current))
                         .map(_.project).distinct.size   
    }).toMap
    
    out.println(dtf_short.print(current) + "," + results_files("JPA")    + "," + results_files("JDBC")    + "," + results_files("Hibernate") + "," +
                                       results_projects("JPA") + "," + results_projects("JDBC") + "," + results_projects("Hibernate"))
    current = current plus Months.ONE
}

out.close()

out: PrintWriter = java.io.PrintWriter@1c734f10
current: DateTime = 2015-04-01T00:00:00.000Z
dtf: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@613c025d

In [224]:
val out = new PrintWriter(new File(directory, "data/partition-ratio-evolution-bis.csv"))
out.println("project,t1,t2,time,t1-nfiles,t2-nfiles")
val interesting_weeks = 0 to 15
val technologies = Seq("JDBC", "JPA", "Hibernate")

technologies.foreach(t1 => {
    technologies.foreach(t2 => {
        // Selection of files in which start_T1 < start_T2 <= end_T1
        val selected_files = entries.filter(file => {
            val start_t1 = file.technologies.get(t1).map(_.getStart)
            val start_t2 = file.technologies.get(t2).map(_.getStart)
            val end_t1   = file.technologies.get(t1).map(_.getEnd)
            
            start_t1.isDefined && start_t2.isDefined && end_t1.isDefined && 
            (start_t1.get < start_t2.get) && (start_t2.get <= end_t1.get)
        })
        
        println(t1 + " " + t2 + " " + selected_files.map(_.project).distinct.size)
        
    })
})

out.close

JDBC JDBC 0
JDBC JPA 13
JDBC Hibernate 4
JPA JDBC 27
JPA JPA 0
JPA Hibernate 13
Hibernate JDBC 10
Hibernate JPA 12
Hibernate Hibernate 0


out: PrintWriter = java.io.PrintWriter@e80814a
interesting_weeks: collection.immutable.Range.Inclusive = Range(
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
...
technologies: Seq[String] = List("JDBC", "JPA", "Hibernate")

Determines the begining and the end of each technology for each project

In [13]:
val project_technologies = scala.collection.mutable.Map[String, scala.collection.mutable.Map[String, Interval]]()
entries.map(_.project).distinct.foreach(project => {
    project_technologies.put(project, scala.collection.mutable.Map.empty)
})

entries.foreach(entry => {
    val project = entry.project
    val current = project_technologies(project)
    entry.technologies.foreach(e => {
        val nInterval = current.getOrElse(e._1, e._2)
        current.put(e._1, new Interval(minDate(nInterval.start, e._2.start), maxDate(nInterval.end, e._2.end)))
    })
})

project_technologies: collection.mutable.Map[String, collection.mutable.Map[String, Interval]] = Map(
  "dekellum_iudex" -> Map("JDBC" -> 2008-12-01T03:50:06.000Z/2015-02-13T22:27:29.000Z),
  "puniverse_galaxy" -> Map("JDBC" -> 2012-07-05T00:19:18.000Z/2015-01-05T00:34:31.000Z),
  "psylock_Diaketas-Manage" -> Map("JDBC" -> 2012-05-14T00:26:17.000Z/2012-06-13T13:26:21.000Z),
  "difi_oxalis" -> Map("JDBC" -> 2013-02-10T22:29:53.000Z/2015-03-16T18:38:20.000Z),
  "amorawski_studies" -> Map("JPA" -> 2012-05-23T18:49:56.000Z/2012-05-28T19:07:56.000Z),
  "seadas_seadas" -> Map("JDBC" -> 2015-02-23T14:09:52.000Z/2015-03-17T17:27:20.000Z),
  "computerlove_Duskenmedarbeidere" -> Map(
    "JDBC" -> 2012-01-29T16:56:05.000Z/2012-01-31T20:31:58.000Z,
...

In [13]:
val technologies = Seq("JPA", "Hibernate", "JDBC")

val out = new PrintWriter(new File(directory, "data/technology-transition.csv"))

out.println("project,t1,t2,time,n_t1,n_t2,residual,suppression,completion,replacement,coverage," +
        "rel_residual,rel_suppression,rel_completion,rel_replacement,rel_coverage")

technologies.foreach(t1 => {
    technologies.foreach(t2 => {
        if(t1 != t2)
        {
           val having_t1_and_t2 = project_technologies.filter(e => {
                (e._2 contains t1) && (e._2 contains t2)
           })
        
           val sandwich = having_t1_and_t2.filter(e => {
                val period_t1 = e._2(t1)
                val period_t2 = e._2(t2)

                (period_t1.start < period_t2.start) && (period_t2.start <= period_t1.end)
           })

           sandwich.foreach(project => {
               val name = project._1
               val introduction_date = project._2(t2).start
               val initial_files = entries.filter(file => {
                   (file.project == name) &&
                   (file.technologies contains t1) &&
                   ((file.technologies(t1) contains introduction_date) || (file.technologies(t1).end == introduction_date))
               }).toSet
               
               val nFiles = initial_files.size.toFloat
               
               val offsets = (0 to 15)
               
               
               offsets.foreach(offset => {
                   val date = introduction_date + offset.weeks
                   if(project_end(name) >= date)
                   {
                       val t1_set = initial_files.filter(file => file.involves(t1, date))
                       val t2_set = initial_files.filter(file => file.involves(t2, date))

                       val residual    = (t1_set -- t2_set).size               // A and not(B)
                       val suppression = ((initial_files -- t1_set) -- t2_set).size  // not(A) and not(B)
                       val completion  = (t1_set intersect t2_set).size           // A and B
                       val replacement = (t2_set -- t1_set).size              // not(A) and B
                       val coverage    = (t1_set union t2_set).size           // A or B

                       val data = Seq(residual, suppression, completion, replacement, coverage)


                       out.println(name + "," + t1 + "," + t2 + "," + offset + "," + t1_set.size + "," + t2_set.size + "," + 
                                   data.mkString(",") + "," +
                                   data.map(_.toFloat / nFiles).mkString(",")
                       )
                   }
               })
           })
        }
    }) 
})

out.close

technologies: Seq[String] = List("JPA", "Hibernate", "JDBC")
out: PrintWriter = java.io.PrintWriter@f0f60e0

In [15]:
val technologies = Seq("JPA", "Hibernate", "JDBC")

technologies.foreach(t1 => {
  technologies.foreach(t2 => {
      if(t1 != t2){
          val having_t1_and_t2 = project_technologies.filter(e => {
                (e._2 contains t1) && (e._2 contains t2)
           })
        
           val sandwich = having_t1_and_t2.filter(e => {
                val period_t1 = e._2(t1)
                val period_t2 = e._2(t2)

                (period_t1.start < period_t2.start) && (period_t2.start <= period_t1.end)
           })
                    
          val res = sandwich.map(project => {
              val t2_start = project._2(t2).start
              
              val candidates = entries.filter(e => {
                  (e.project == project._1) && 
                  (e.involves(t1, t2_start))
              })
              
              val selected = candidates.count(c => c.involves(t2))
              
              (selected, candidates.size)
          }).reduce((a,b) => (a._1 + b._1 , a._2 + b._2))
          
          println(t1 + "," + t2 + "," + res._1 + "," + res._2)
      }
  })  
})

JPA,Hibernate,13,200
JPA,JDBC,10,53
Hibernate,JPA,44,446
Hibernate,JDBC,6,55
JDBC,JPA,27,316
JDBC,Hibernate,8,108


technologies: Seq[String] = List("JPA", "Hibernate", "JDBC")